In [ ]:
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import utils.config as cfg_
import time
import seaborn as sns

In [ ]:
#Get W&B API
api = wandb.Api()

In [ ]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="05-xai-lrp-TEST"
runname = name
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"

In [ ]:
config_lrp = cfg_.get_artifact_config_xai_lrp(False)
cfg_.show_attrdict(config_lrp)

In [ ]:
config_lrp.use_wandb

In [ ]:
run_lrp = wandb.init(
    entity           = config_lrp.wandb_entity,
    project          = config_lrp.wandb_project if config_lrp.use_wandb else 'work-nbs', 
    group            = config_lrp.wandb_group,
    allow_val_change = config_lrp.allow_val_change, 
    job_type         = config_lrp.job_type, 
    mode             = 'online' if config_lrp.use_wandb else 'disabled',
    anonymous        = 'never' if config_lrp.use_wandb else 'must',
    config           =  config_lrp,
    resume           = 'allow',
    name = runname
)
config_lrp = wandb.config # Object for storing hyperparameters

In [ ]:
embs, emb_artifact, emb_config = get_embeddings(config_lrp, run_lrp, api, True)

In [ ]:
embs.shape

In [ ]:
cfg_.show_attrdict(emb_config)

In [ ]:
#| export
def get_dataset_old(config, run_dr, config_dr, api, print_flag = False):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')
    if print_flag: print (enc_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner

In [ ]:
#| export
def get_dataset(
    config_lrp, 
    config_emb,
    config_dr,
    run_lrp,
    api, 
    print_flag = False
):
    # Botch to use artifacts offline
    artifacts_gettr = run_lrp.use_artifact if config_lrp.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config_emb['enc_artifact'], type='learner')
    if print_flag: print (enc_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    #cfg_.show_attrdict(enc_logger.config)
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    
    if config_dr['dr_artifact'] is not None:
        print("Is not none")
        dr_artifact = artifacts_gettr(config_dr['enc_artifact'])
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    if print_flag: print("--> DR artifact name", dr_artifact.name)
    dr_artifact
    df = dr_artifact.to_df()
    if print_flag: print("--> DR After to df", df.shape)
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner

In [ ]:
cfg_.show_attrdict(emb_config)

In [ ]:
config_dr, _ = cfg_.get_artifact_config_dimensionality_reduction(False)

In [ ]:
cfg_.show_attrdict(config_dr)

In [ ]:
df, dr_artifact, enc_artifact, enc_learner = get_dataset(
    config_lrp,
    emb_config,
    config_dr,
    run_lrp,
    api, 
    True
)

In [ ]:
artifacts_gettr = run_lrp.use_artifact if config_lrp.use_wandb else api.artifact

In [ ]:
enc_artifact = artifacts_gettr(emb_config['enc_artifact'], type='learner')

In [ ]:
enc_artifact.name

In [ ]:
embs

In [ ]:
config_dr

In [ ]:
df, dr_artifact, enc_artifact, enc_learner = get_dataset(
    config_dr, 
    run_lrp, 
    emb_config, 
    

In [ ]:
enc_artifact = artifacts_gettr(emb_config['enc_artifact'], type='learner')

In [ ]:
enc_artifact

In [ ]:
enc_logger = enc_artifact.logged_by()

In [ ]:
enc_learner = enc_learner.logged_by()

In [ ]:
def get_embeddings(config, run_dr, config_dr, api, print_flag = False):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.emb_artifact, type='object')
    if print_flag: print (emb_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner